In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
df_train=pd.read_csv('../input/cassandra-udyam-2022/trainData.csv')

In [ ]:
df_train.head()

In [ ]:
df_train.dtypes

In [ ]:
df_train.describe() 

In [ ]:
len(df_train)

In [ ]:
df_train.isnull().sum()

In [ ]:
coun_t  = df_train.nunique().to_frame(name = 'Counts')
coun_t

In [ ]:
from datetime import time, date, datetime

In [ ]:
def convertInt(df):
  df['Amount'] = df['Amount'].astype('int')
  df['Settled'] = df['Settled'].astype('int')
  df['Outstanding_1'] = (df['Amount'] - df['Settled']).astype('int')
  df.drop(['Outstanding'], axis=1, inplace=True)
  return df

In [ ]:
def convertDate(df):
  df['Hours'] = pd.to_datetime(df['Created']).dt.time
  df['Hours'] = df.Hours.apply(lambda x: x.hour)
  df['Hours']=df['Hours'].astype(int)

  df['Invoice_Date'] = pd.to_datetime(df['Invoice_Date']).dt.date
  df['Due_Date'] = pd.to_datetime(df['Due_Date']).dt.date
  df['Diff_Date'] = (df['Due_Date'] - df['Invoice_Date']).dt.days
  
  return df

In [ ]:
def processDate(df):
    print(df.info())
    # df['Date'] = df['Date'].apply(lambda x:datetime.strptime(x, "%d/%m/%Y").strftime("%Y-%m-%d"))
    df['Created'] = pd.to_datetime(df['Created'])
    df['year_c'] = pd.to_datetime(df['Created']).dt.year
    df['month_c'] = pd.to_datetime(df['Created']).dt.month
    df['day_c'] = df['Created'].apply(lambda x : x.dayofweek)
    df.drop(['Created'], axis=1, inplace=True)
    
    df['Invoice_Date'] = pd.to_datetime(df['Invoice_Date'])
    df['year_i'] = pd.to_datetime(df['Invoice_Date']).dt.year
    df['month_i'] = pd.to_datetime(df['Invoice_Date']).dt.month
    df['day_i'] = df['Invoice_Date'].apply(lambda x : x.dayofweek)
    df.drop(['Invoice_Date'], axis=1, inplace=True)

    df['Due_Date'] = pd.to_datetime(df['Due_Date'])
    df['year_d'] = pd.to_datetime(df['Due_Date']).dt.year
    df['month_d'] = pd.to_datetime(df['Due_Date']).dt.month
    df['day_d'] = df['Due_Date'].apply(lambda x : x.dayofweek)
    df.drop(['Due_Date'], axis=1, inplace=True)
    return df

In [ ]:
def neg_values(df):
  indexNames = df[ df['Number_of_Days_until_Payment'] < 0 ].index
  df.drop(indexNames , inplace=True)
  return df

In [ ]:
def getDataframe(Vendor_Name, pred):
    sub=pd.DataFrame([Vendor_Name, pred])
    sub=sub.transpose()
    sub.columns=['Vendor_Name', 'Number_of_Days_until_Payment']
    sub['Number_of_Days_until_Payment'] = sub['Number_of_Days_until_Payment'].apply(np.floor)
    sub['Vendor_Name']=sub['Vendor_Name'].astype(object)
    sub.to_csv('submission.csv', index=False)
    
    return sub

In [ ]:
convertDate(df_train)
processDate(df_train)
convertInt(df_train)
neg_values(df_train)

In [ ]:
df_train.head()

In [ ]:
df_train.drop('Description', axis = 1, inplace = True)

In [ ]:
label=df_train['Number_of_Days_until_Payment']
print(label)

In [ ]:
x_train=df_train
# x_train.drop(['Number_of_Days_until_Payment'], axis=1, inplace=True)
x_train.drop(['Vendor_Name', 'Number_of_Days_until_Payment' ], axis=1, inplace=True)
x_train.head()

In [ ]:
DF=pd.read_csv('../input/cassandra-udyam-2022/testData.csv')
X_test=DF
data=DF
Vendor_Name =X_test['Vendor_Name'].astype(object)
X_test.drop(['Description'], axis=1, inplace=True)
X_test.drop(['Vendor_Name'], axis=1, inplace=True)

In [ ]:
Vendor_Name

In [ ]:
X_test.head()

In [ ]:
X_test = convertDate(X_test)
X_test=processDate(X_test)
X_test = convertInt(X_test)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler


In [ ]:
x1, x2, y1, y2 = train_test_split(x_train, label, test_size=0.1, random_state=42)

sc = StandardScaler()
X_train_std = sc.fit_transform(x1)#doubt#
# X_test_std = sc.transform(x2)
X_test_std = sc.transform(x2)

In [ ]:
gbr_params = {'n_estimators': 1000,
          'max_depth': 15,
          'min_samples_split': 5,
          'learning_rate': 0.1,
          'loss': 'squared_error'}

gbr = GradientBoostingRegressor(**gbr_params)
gbr.fit(X_train_std, y1)

print(gbr.score(X_test_std, y2))
X_std=sc.transform(X_test)
pred=gbr.predict(X_std)
# pred = pred.astype(int)
print(getDataframe(Vendor_Name, pred))